In [1]:
from newsapi import NewsApiClient
import pandas as pd
import numpy as np1
from datetime import datetime, timedelta
import requests
import json
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

load_dotenv()


False

In [2]:
newsapi = NewsApiClient(api_key="4e422ec24e774982ad908da018eb48f7")
data = newsapi.get_sources()

# Classify sources by category
categories = {}
countries = {}

In [3]:
for source in data['sources']:
    category = source['category']
    country = source['country']
    
    if category not in categories:
        categories[category] = []
    categories[category].append(source)
    
    if country not in countries:
        countries[country] = []
    countries[country].append(source)

def create_dataframe(data_dict, outer_key_name, inner_key_name):
    records = []
    for outer_key, sources in data_dict.items():
        for source in sources:
            record = {
                outer_key_name: outer_key,
                inner_key_name: source['name'],
                'id': source['id'],
                'description': source['description'],
                'url': source['url'],
                'category': source['category'],
                'language': source['language'],
                'country': source['country']
            }
            records.append(record)
    return pd.DataFrame(records).set_index([outer_key_name, inner_key_name])

In [41]:
# Create DataFrame for categories
df_categories = create_dataframe(categories, 'Category', 'Source Name')
yesterday_date = (datetime.now() - timedelta(days=10)).strftime('%Y-%m-%d')
df_categories['yesterday_date'] = yesterday_date
df_categories.drop(labels='category', axis=1, inplace=True)

# Create DataFrame for countries
df_countries = create_dataframe(countries, 'Country', 'Source Name')

list_sources = ['abc-news', 'al-jazeera-english', 'bbc-news', 'google-news', 'google-news-in', 'politico', 'reuters', 'reddit-r-all', 'the-hindu', 'bloomberg', 
                'the-times-of-india', 'business-insider', 'vice-news', 'the-wall-street-journal', 'wired', 'national-geographic', 'bbc-sport', 'espn', 'techcrunch',
                'crypto-coins-news','buzzfeed','entertainment-weekly','medical-news-today','fortune','new-scientist','the-lad-bible', 'new-york-magazine', 'the-washington-times',
                'the-washington-post', 'cnn', 'ign', 'the-verge', 'fox-sports', 'talksport', 'the-sport-bible', 'hacker-news', 'recode', 'the-next-web', 'entertainment-weekly',
                'time', 'the-hindu', 'cbs-news', 'asutralian-financial-review', 'business-insider-uk', 'financial-post', 'fortune', 'info-money']

df_categories = df_categories[df_categories['id'].isin(list_sources)]

In [5]:
def fetch_json(source_id, date):
    url = (f'https://newsapi.org/v2/everything?'
           f'sources={source_id}&'
           f'from={date}&'
           'sortBy=popularity&'
           'apiKey=4e422ec24e774982ad908da018eb48f7')
    
    response = requests.get(url)
    return response.json()


In [17]:
def scrape_content(urls):
    contents = []
    for url in urls:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            article_content = soup.find_all('p')
            content_text = ' '.join([paragraph.get_text() for paragraph in article_content])
            contents.append(content_text)
        except Exception as e:
            contents.append(f"Error: {str(e)}")
    return contents


In [18]:
def json_to_df(data):
    try:
        articles = data['articles']
        if not articles:
            raise ValueError("No articles found for the given date.")
        
        df = pd.DataFrame(articles)
        df = df[['author', 'title', 'url', 'description', 'publishedAt']]
        df.columns = ['author', 'title', 'url', 'description', 'date']
        
        # Adding the content column by scraping the URLs
        df['content'] = scrape_content(df['url'])
        
        return df
    except KeyError as e:
        print(f"KeyError: {e}")
        return pd.DataFrame()
    except ValueError as e:
        print(f"ValueError: {e}")
        return pd.DataFrame()

In [19]:
# Function to remove rows with empty content
def remove_empty_content(df):
    if 'content' in df.columns:
        # Removing rows with empty content
        df = df[df['content'].str.strip().astype(bool)]
        df = df.reset_index(drop=True)
    return df

In [47]:
n = 12
df_trial = df_categories.iloc[[n], :].copy()
df_trial

,,id,description,url,language,country,yesterday_date
Category,Source Name,,,,,,
general,The Times of India,the-times-of-india,Times of India brings the Latest News and Top ...,http://timesofindia.indiatimes.com,en,in,2024-06-06


In [48]:
df_trial = df_categories.iloc[[n], :].copy()
df_trial['data'] = df_trial.apply(
    lambda row: json_to_df(fetch_json(row['id'], row['yesterday_date'])),
    axis=1
)

# Apply the function to clean up each DataFrame in the 'data' column
df_trial['data'] = df_trial['data'].apply(remove_empty_content)
# Remove rows where 'data' column has empty DataFrames
df_trial1 = df_trial[df_trial['data'].apply(lambda x: not x.empty)].copy()

In [54]:
text = df_trial1['data'][0]['content'][5]

C:\Users\abofm\AppData\Local\Temp\ipykernel_17088\3725325152.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = df_trial1['data'][0]['content'][5]


In [55]:
# Replace each period with a period followed by a newline character
formatted_text = text.replace('.', '.\n')

# Define the filename
filename = 'output.txt'

# Open the file in write mode and save the formatted text
with open(filename, 'w',encoding='utf-8') as file:
    file.write(formatted_text)

print(f"The text has been saved to {filename}")

The text has been saved to output.txt


In [43]:
df_trial

,,id,description,url,language,country,yesterday_date,data
Category,Source Name,,,,,,,
general,The Hindu,the-hindu,"The Hindu. latest news, analysis, comment, in-...",http://www.thehindu.com,en,in,2024-06-06,Empty DataFrame Columns: [] Index: []
